In [ ]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader,random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# path configuration
root = 'gdrive/My Drive/project/'
LABELS_PATH = root+'data/MSVD_label_final.csv'
DATA_PATH = root+'data/MSVD/training_data/feat/'
MODEL_PATH = root+'model/'

## Dataset

In [ ]:
label_final_df = pd.read_csv(LABELS_PATH)
label_final_df.tail(5)

,caption,video_id,sent_len
19714,a man and two women walk across the beach,jLgmCY1fEE8_16_26.avi,9
19715,criss angel talking to three women,jLgmCY1fEE8_16_26.avi,6
19716,chris angel gathering women on the beach,jLgmCY1fEE8_16_26.avi,7
19717,criss angel talks to women on the beach,jLgmCY1fEE8_16_26.avi,8
19718,the man is talking to women,jLgmCY1fEE8_16_26.avi,6


### 1. Load Embedding

In [ ]:
embedding = np.load(MODEL_PATH+'MSVD_embedding.npy')
word2idx = pickle.load(open(MODEL_PATH+'MSVD_word2idx.pkl', "rb"))
idx2word = pickle.load(open(MODEL_PATH+'MSVD_idx2word.pkl', "rb"))

In [ ]:
vid = 'jLgmCY1fEE8_16_26.avi'

In [ ]:
filename = DATA_PATH + f'{vid}.npy'

### 2.Dataset Class for pytorch

In [ ]:
class MSVD(Dataset):
    def __init__(self, df, word2idx, DATA_PATH):
        super(MSVD, self).__init__()
        self.df = df
        self.path = DATA_PATH
        self.word2idx = word2idx
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        vid = row['video_id']
        filename = self.path + f'{vid}.npy'
        x = torch.tensor(np.load(filename))
        sentence_emb = [self.word2idx.get(word, self.word2idx['<UNK>']) for word in row['caption'].split(' ')]
        y = torch.zeros(len(sentence_emb)+2)
        y[0], y[-1] = self.word2idx['<START>'], self.word2idx['<END>']
        y[1:-1] = torch.tensor(sentence_emb)
        #true_sentence = row['caption']
        return x.float(), y.long()#, true_sentence

ds = MSVD(label_final_df, word2idx, DATA_PATH)

In [ ]:
ds_iter = iter(ds)

In [ ]:
x, y = next(ds_iter)
x, y

(tensor([[0.0000, 0.6935, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.5050, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.7072, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0081, 1.7673, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.7900, 1.6565, 0.0000,  ..., 0.2572, 0.0000, 0.0000],
         [0.0000, 2.0024, 0.0000,  ..., 0.8096, 0.0000, 0.0000]]),
 tensor([  1,   4,   8, 782, 580,   4,  48,   2]))

In [ ]:
[idx2word[word.item()] for word in y]

['<START>', 'a', 'woman', 'goes', 'under', 'a', 'horse', '<END>']

### 3.Dataloaders

In [ ]:
def collate_fn(batch):
    """
    padding  text or video
    """
    x_batch = []
    y_batch = []
    for x, y in batch:
        x_batch.append(x)
        y_batch.append(y)
    lengths = list(map(lambda x: x.size(0), x_batch))
    x_batch = torch.nn.utils.rnn.pad_sequence(x_batch,  batch_first=True)
    x_batch = torch.nn.utils.rnn.pack_padded_sequence(x_batch, lengths, batch_first=True, enforce_sorted=False)
    y_batch = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True)
    return x_batch, y_batch#, lengths  # PackedSequence, padded tensor

In [ ]:
# split train/test
train_proportion = 0.90
train_size = int(train_proportion * len(ds))
validation_size = len(ds) - train_size
print(train_size, validation_size)

17747 1972


In [ ]:
train_ds, validation_ds = random_split(ds, [train_size, validation_size])
# dataloaders
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(validation_ds, batch_size=256, shuffle=False, collate_fn=collate_fn)

In [ ]:
loader = iter(train_dl)

In [ ]:
x, y = next(loader)
x, y

(PackedSequence(data=tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2822, 1.7576],
        [0.0000, 1.6779, 0.0000,  ..., 0.0000, 0.4317, 0.0000],
        [0.0000, 0.0000, 3.9638,  ..., 0.0000, 0.0000, 0.8791],
        ...,
        [1.1512, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0156],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0527, 0.7188],
        [3.0851, 0.0000, 0.0000,  ..., 0.0189, 0.0000, 0.3621]]), batch_sizes=tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16]), sorted_indices=tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]), unsorted_indices=tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])),
 tensor([[   1,   19,   8

# Model

### 1.Sequence to Sequence Model (Encoder-Decoder)

While image description handles a variable length output sequence of words, video description also has to handle a variable length input sequence of frames. 
Related approaches to video description have resolved variable length input by holistic video representations [29, 28, 11], pooling over frames [39], or sub-sampling on a fixed number of input frames [43]. In contrast, in this work we propose a sequence to sequence

In [ ]:
class Encoder(nn.Module):
    '''
    Take sequence of video's resnet50 features as input
    
    note: batch_first=True does not apply to hidden or cell states
    '''
    def __init__(self, input_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)
                
    def forward(self, x):
        '''
        x: PackedSequence
        '''
        outputs, hidden = self.rnn(x) 
        return hidden
    
class Decoder(nn.Module):
    '''
    Decode Hidden State from Encoder to sentence (sequence of texts)
    
    note: batch_first=True does not apply to hidden or cell states
    '''
    def __init__(self, weights, emb_dim, hidden_dim, out_dim):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.emb_dim = emb_dim
        self.out_dim = out_dim
        
        # layers
        #self.emb = nn.Embedding.from_pretrained(torch.tensor(weights), padding_idx=0, freeze=False)
        self.emb = nn.Embedding(out_dim, emb_dim, padding_idx=0)
        self.rnn = nn.GRU(emb_dim + hidden_dim, hidden_dim, num_layers=1, batch_first=True)
        self.fc_out1 = nn.Linear(emb_dim + hidden_dim * 2, out_dim)
        self.fc_out2 = nn.Linear(out_dim, out_dim)

                
    def forward(self, word_input, encoded_context, hidden):
        '''
        word_input: (batch_size)
        encoded_context: (1, batch_size, hidden_dim)
        hidden: (1, batch_size, hidden_dim)
        '''
        # 1 word at a time
    
        word_input = self.emb(word_input) # dim (batch, emb_dim) 
        emb_input = torch.cat([word_input, encoded_context.squeeze(0)], dim=1)
        output, hidden = self.rnn(emb_input.unsqueeze(1).float(), hidden)
        prediction = F.relu(self.fc_out1(torch.cat([word_input, encoded_context.squeeze(0), hidden.squeeze(0)], dim=1).float()))
        prediction = self.fc_out2(prediction)
        return prediction, hidden 
    

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, x, y, teacher_forcing_ratio=0.8):
        '''
        x: PackedSequence
        y: (batch_size, sentence_len(padded))
        hidden: (1, batch_size, hidden_dim)
        '''
        batch_size = y.size(0)
        sentence_len = y.size(1)
        vocab_size = self.decoder.out_dim
        
        ##############
        # Initialize #
        ##############
        # tensor for final outputs
        outputs = torch.zeros(batch_size, sentence_len, vocab_size).to(self.device)
        # last hidden state of the encoder is the context
        encoded_context = self.encoder(x) # (1, batch_size, hidden_dim)
        # first hidden state 
        hidden = encoded_context # (1, batch_size, hidden_dim)
        # first input '<START>'
        word_input = y[:, 0] # (batch_size)
        for t in range(1, sentence_len):
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(word_input, encoded_context, hidden)
            
            #place predictions in a tensor holding predictions for each token
            outputs[:, t, :] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = np.random.rand() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) # dim: (batch_size)
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            word_input = y[:, t] if teacher_force else top1

        return outputs
    

# Training 

Set up the model

In [ ]:
N_VOCAB = len(word2idx)
EMB_DIM = 300
INPUT_DIM = 4096 # vgg16 fc dim
HIDDEN_DIM = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = Encoder(INPUT_DIM, HIDDEN_DIM)
decoder = Decoder(embedding, EMB_DIM, HIDDEN_DIM, N_VOCAB)
model = Seq2Seq(encoder, decoder, device)
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (rnn): GRU(4096, 512, batch_first=True)
  )
  (decoder): Decoder(
    (emb): Embedding(1504, 300, padding_idx=0)
    (rnn): GRU(812, 512, batch_first=True)
    (fc_out1): Linear(in_features=1324, out_features=1504, bias=True)
    (fc_out2): Linear(in_features=1504, out_features=1504, bias=True)
  )
)

weight initialization with N(0, 0.01)

In [ ]:
#for name, param in model.named_parameters():
#    #if name != 'decoder.emb.weight':
#    print(name, param.shape)
#    nn.init.normal_(param.data, mean=0, std=0.01)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_func = nn.CrossEntropyLoss(ignore_index=0)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, threshold=0.001, threshold_mode='rel', min_lr=0)

In [ ]:
def train_one_epoch(model, dataloader, optimizer, lossFun, backwards=True, print_loss=False):
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    
    for i, (x, y) in enumerate(tqdm(dataloader)):
        x = x.to(device)
        y = y.to(device)
        out = model(x, y)
        out = out.view(-1, N_VOCAB)
        y = y.view(-1)
        loss = lossFun(out, y)
        
        total_loss += loss.item()
        if i % 10 == 0:
            print(f'Batch loss: {loss.item()}')
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
    
    avg_loss = total_loss / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
    
    return avg_loss



def evaluate(model, dataloader, lossFun):
    
    model.eval()
    
    total_loss = 0
    
    with torch.no_grad():
    
        for x, y in tqdm(dataloader):
            x = x.to(device)
            y = y.to(device)
            out = model(x, y)
            out = out.view(-1, N_VOCAB)
            y = y.view(-1)
            loss = lossFun(out, y)

            total_loss += loss.item()
        
    return total_loss / len(dataloader)

In [ ]:
device

device(type='cuda')

In [ ]:
# train 3 more epochs
EPOCHS = 100
best = 100000
train_losses = []
valid_losses = []
for epoch in range(EPOCHS):
    print('Epoch: ', epoch)
    
    train_loss = train_one_epoch(model, train_dl, optimizer, loss_func)
    print('Train Loss: ', train_loss)
    train_losses.append(train_loss)
    
    valid_loss = evaluate(model, valid_dl, loss_func)
    print('Valid Loss: ', valid_loss)
    valid_losses.append(valid_loss)

    if valid_loss < best:
        best = valid_loss
        torch.save(model.state_dict(),  MODEL_PATH+'MSVD_seq2seq_v2.pt')
    scheduler.step(valid_loss)



  0%|          | 0/70 [00:00<?, ?it/s]

Epoch:  0




  1%|▏         | 1/70 [01:47<2:04:04, 107.90s/it]

Batch loss: 7.318040370941162




  3%|▎         | 2/70 [03:16<1:55:40, 102.06s/it]

  4%|▍         | 3/70 [04:30<1:44:40, 93.75s/it] 

  6%|▌         | 4/70 [05:30<1:31:56, 83.59s/it]

  7%|▋         | 5/70 [06:27<1:21:53, 75.59s/it]

  9%|▊         | 6/70 [07:12<1:10:48, 66.39s/it]

 10%|█         | 7/70 [07:43<58:26, 55.66s/it]  

 11%|█▏        | 8/70 [08:13<49:33, 47.96s/it]

 13%|█▎        | 9/70 [08:38<41:46, 41.09s/it]

 14%|█▍        | 10/70 [09:00<35:36, 35.60s/it]

 16%|█▌        | 11/70 [09:20<30:21, 30.87s/it]

...

Batch loss: 1.3293395042419434




 89%|████████▊ | 62/70 [02:29<00:19,  2.42s/it]

 90%|█████████ | 63/70 [02:32<00:17,  2.43s/it]

 91%|█████████▏| 64/70 [02:34<00:14,  2.44s/it]

 93%|█████████▎| 65/70 [02:36<00:12,  2.44s/it]

 94%|█████████▍| 66/70 [02:39<00:09,  2.46s/it]

 96%|█████████▌| 67/70 [02:41<00:07,  2.44s/it]

 97%|█████████▋| 68/70 [02:44<00:04,  2.45s/it]

 99%|█████████▊| 69/70 [02:46<00:02,  2.45s/it]

100%|██████████| 70/70 [02:47<00:00,  2.39s/it]


  0%|          | 0/8 [00:00<?, ?it/s]

Train Loss:  1.4576427068029132




 12%|█▎        | 1/8 [00:02<00:17,  2.46s/it]

 25%|██▌       | 2/8 [00:04<00:14,  2.44s/it]

 38%|███▊      | 3/8 [00:07<00:11,  2.38s/it]

 50%|█████     | 4/8 [00:09<00:09,  2.35s/it]

 62%|██████▎   | 5/8 [00:11<00:06,  2.33s/it]

 75%|███████▌  | 6/8 [00:14<00:04,  2.35s/it]

 88%|████████▊ | 7/8 [00:16<00:02,  2.35s/it]

100%|██████████| 8/8 [00:17<00:00,  2.24s/it]

Valid Loss:  2.3621239066123962
